In [5]:
import time
import json
import serial

class EEGSDK:
    def __init__(self, serial_port, baud_rate=9600):
        self.serial_port = serial_port
        self.baud_rate = baud_rate
        self.serial_device = None
        self.serialized_data = []
        

    def connect(self):
        try:
            self.serial_device = serial.Serial(self.serial_port, self.baud_rate)
            print(f"Connected to EEG hardware via {self.serial_port}.")

        except serial.SerialException as e:
            print(f"Error connecting to EEG hardware: {e}")

    def disconnect(self):
        if self.serial_device is not None:
            self.serial_device.close()
            print("Disconnected from EEG hardware.")
        else:
            print("Not connected to EEG hardware.")

    def start_data_acquisition(self):
        try:
            self.serial_device.write(b'START\n')  # Add a newline character
            print("Started data acquisition.")

        except serial.SerialException as e:
            print(f"Error starting data acquisition: {e}")

    def stop_data_acquisition(self):
        try:
            self.serial_device.write(b'STOP\n')  # Add a newline character
            print("Stopped data acquisition.")

        except serial.SerialException as e:
            print(f"Error stopping data acquisition: {e}")

    def retrieve_eeg_data(self):
        try:
            while True:
                data = self.serial_device.readline().decode('utf-8')
                data_point = json.loads(data)
                yield data_point
                time.sleep(1)

        except serial.SerialException as e:
            print(f"Error retrieving EEG data: {e}")

    def to_json(self):
        return json.dumps({"status": "success", "message": "EEG SDK is operational"})



In [ ]:
# Example usage:
if __name__ == "__main__":
    
    eeg_sdk = EEGSDK(serial_port='COM10', baud_rate=115300)
    eeg_sdk.connect()
    eeg_sdk.start_data_acquisition()

    try:
        for data_point in eeg_sdk.retrieve_eeg_data():
            print(data_point)
            # Process the data as needed

    except KeyboardInterrupt:
        # Stop data acquisition on keyboard interrupt
        eeg_sdk.stop_data_acquisition()
        eeg_sdk.disconnect()
